# Hypothesis testing

We have two populations X1 and X2, each of which we take a sample of (size n for X1, m for for X2).

We want to test whether their population means are significantly different.

**Hypotheses (two-sided):**

- H_0: mu_X1-X2 = 0
- H_1: mu_X1-X2 != 0

**Assuming:**

- unknown distributions X1, X2
- sigma_X1 not necessarily equal to sigma_X2

## Define functions

In [1]:
import numpy as np
from scipy.stats import norm, t



def sample_statistics(sample):
    """Calculates sample statistics from sample
    
    Args:
        sample (np.array): 1-dimensional array with numeric sample values
    
    Returns:
        (int, float, float): Tuple of sample size, sample mean, sample standard deviation
    """
    n = len(sample)
    mean = sum(sample) / n
    s = np.sqrt(sum((sample - mean) ** 2) / (n - 1))
    
    return n, mean, s

assert sample_statistics(np.array([1.1, 5.22, 8.25, 3.00, 3.39, 3.89, 2.2])) == (7, 3.8642857142857143, 2.3258034965925014)



def zscore_critical(p, twosided = False):
    """Calculates z-score from confidence level p
    
    Args:
        p (float): Confidence level
        twosided (bool): True for twosided confidence interval, False for onesided

    Returns:
        float: z-score
    """
    if not (p >= 0 and p <=1):
        raise ValueError("Invalid value for p")
    
    if twosided:
        return norm.ppf((1 + p) / 2)
    else:
        return norm.ppf(p)    
        
assert (zscore_critical(p=0.95, twosided=True) == 1.959963984540054)
assert (zscore_critical(p=0.5, twosided=False) == 0.0)
assert (zscore_critical(p=0.95, twosided=False) == 1.6448536269514722)



def tscore_critical(p, degrees, twosided = False):
    """Calculates t-score from confidence level p and degrees of freedom
    
    Args:
        p (float): Confidence level
        degrees (int): Degrees of freedom
        twosided (bool): True for twosided confidence interval, False for onesided

    Returns:
        float: z-score
    """
    if not (p >= 0 and p <=1):
        raise ValueError("Invalid value for p")
        
    if twosided:
        return t.ppf((1 + p) / 2, degrees)
    else:
        return t.ppf(p, degrees)

assert (tscore_critical(p = 0.95, degrees=10, twosided=False) == 1.8124611228107335)
assert (tscore_critical(p = 0.95, degrees = 1000000, twosided=True) == 1.9599663568141066)

## Inputs

We know:
- list of samples for X1: X1_sample
- list of samples for X2: X2_sample
- confidence level: alpha

In [2]:
X1_sample = np.array([1.1, 5.22, 8.25, 3.00, 3.39, 3.89, 2.2, 1])
X2_sample = np.array([5.01, 5.73, 4.99, 8.83, 9.93])
p = 0.95

## Calculations

### Sample statistics and difference of sample means

We derive:
- sample sizes n, m
- sample means X1bar, X2bar
- sample standard deviations s_X1, s_X2
- difference of sample means (statistic of interest!)

In [3]:
# Sample sizes, sample means, sample standard deviations
n, X1bar, s_X1 = sample_statistics(X1_sample)
m, X2bar, s_X2 = sample_statistics(X2_sample)

# Statistic of interest: difference of sample means
Xmeandifference = X1bar - X2bar
print("Difference in sample means: {:f}".format(Xmeandifference))

Difference in sample means: -3.391750


### Sampling distribution of sampling means

We are interested in the distribution of $\bar{X_1}-\bar{X_2}$, i.e. the sampling distribution of differences in sample means.

Under the null hypothesis we have:

- Mean of the difference of the sampling means:
$$\mu_{\bar{X_1}-\bar{X_2}} = 0$$

- Standard deviation of the difference of the sampling means:
$${\sigma_{\bar{X_1}-\bar{X_2}}^2} = {\sigma_{\bar{X_1}}^2} + {\sigma_{\bar{X_2}}^2} = \frac{\sigma_{X_1}^2}{n} + \frac{\sigma_{X_2}^2}{m} \approx \frac{s_{X_1}^2}{n} + \frac{s_{X_2}^2}{m} $$

In the last step we estimated $\sigma_{X_1}$, $\sigma_{X_2}$ to be approximately equal to the sample standard deviations $s_{X_1}$, $s_{X_2}$. This is because we don't know their actual values.

In [7]:
std_Xmeandifference = np.sqrt(s_X1**2 / n + s_X2**2 / m)
std_Xmeandifference

print("Standard deviation of distribution of difference in sample means: {:f}".format(std_Xmeandifference))

Standard deviation of distribution of difference in sample means: 1.335106


### Question

What is the probability of getting this sample value of $X_1-X_2$ under the null hypothesis?

In [5]:
t_statistic = Xmeandifference / std_Xmeandifference
print("t-statistic: {:f}".format(t_statistic))

t-statistic: -2.540435


### Critial z/t-score

In order to determine this we need to calculate the critical t-score (or z-score in case of large sample sizes).

In [6]:
# Get critical t-score (or z-score) from confidence level alpha
t_crit = tscore_critical(p=p, degrees=n+m-1, twosided=True)
print("Critical t-score: {:f}".format(t_crit))

Critical t-score: 2.178813


## Single function to check hypothesis

In [40]:
def check_null_hypothesis_twosided (X1_sample, X2_sample):
    # Sample sizes, sample means, sample standard deviations
    n, X1bar, s_X1 = sample_statistics(X1_sample)
    m, X2bar, s_X2 = sample_statistics(X2_sample)
    print("Sample sizes: {}, {}".format(n, m))
    print("Sample means: {:.4n}, {:.4n}".format(X1bar, X2bar))
    print("Sample standard deviations: {:.4n}, {:.4n}".format(s_X1, s_X2))

    # Statistic of interest: difference of sample means
    Xmeandifference = X1bar - X2bar
    print("Difference in sample means: {:.4n}".format(Xmeandifference))
    
    # Standard deviation of sample mean difference
    std_Xmeandifference = np.sqrt(s_X1**2 / n + s_X2**2 / m)
    std_Xmeandifference
    print("Standard deviation of distribution of difference in sample means: {:.4n}".format(std_Xmeandifference))
    
    # Determine t-statistic
    t_statistic = Xmeandifference / std_Xmeandifference
    print("t-statistic: {:.4n}".format(t_statistic))
    
    # Get critical t-score (or z-score) from confidence level alpha
    t_crit = tscore_critical(p=p, degrees=n+m-1, twosided=True)
    print("Critical t-score: {:.4n}".format(t_crit))

    # Check hypothesis
    if (abs(t_statistic) > abs(t_crit)):
        print("Null hypothesis REJECTED")
        return False
    else:
        print("Null hypothesis NOT rejected")
        return True
    
    
check_null_hypothesis_twosided(X1_sample, X2_sample)

Sample sizes: 8, 5
Sample means: 3.506, 6.898
Sample standard deviations: 2.38, 2.318
Difference in sample means: -3.392
Standard deviation of distribution of difference in sample means: 1.335
t-statistic: -2.54
Critical t-score: 2.179
Null hypothesis REJECTED


False